In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Update the path to include the data directory
amazon_sales = 'data/Amazon Sale Report.csv'

# Now read the CSV file from the data directory
sales = pd.read_csv(amazon_sales)

amazon_sales= 'Amazon Sale Report.csv' 

sales=pd.read_csv(amazon_sales)




#CLEANING: 

sales = sales.drop(columns=['fulfilled-by', 'Unnamed: 22'])


# Function to display initial data overview
def data_overview(df, name):
    print(f"\nOverview of {name} dataset:")
    print(df.info())
    print(df.head())
    print(f"\nNumber of duplicates in {name}: {df.duplicated().sum()}")
    print(f"Missing values in {name}:\n{df.isnull().sum()}\n")
    
# Replace missing values in 'Courier Status' with 'Unknown' since 5.33% are null
sales['Courier Status'].fillna('Unknown', inplace=True)

# Replace missing values in 'currency' with 'INR' since 6.04% are null & all non-null are INR
sales['currency'].fillna('INR', inplace=True)

# Calculate the mean and median of the 'Amount' column 
amount_mean = sales['Amount'].mean()
amount_median = sales['Amount'].median()
# Replace missing values in 'Amount' with the mean since 6.04% are null
sales['Amount'].fillna(amount_mean, inplace=True)

# Replace missing values with 'Unknown'
columns_to_fill = ['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']

for column in columns_to_fill:
    sales[column].fillna('Unknown', inplace=True)

# Replace NaN values in 'promotion-ids' with 'None'
sales['promotion-ids'] = sales['promotion-ids'].fillna('None')


sales['Date'] = pd.to_datetime(sales['Date'], errors='coerce')
sales['Fulfilment'] = sales['Fulfilment'].astype('category')
sales.columns = sales.columns.str.strip()

# List of columns to convert to 'category' type
columns_to_convert = ['Fulfilment', 'Sales Channel', 'ship-service-level']

# Convert the columns to 'category' type
sales[columns_to_convert] = sales[columns_to_convert].astype('category')
     
    
# Display initial data overviews
data_overview(sales, "sales")



FileNotFoundError: [Errno 2] No such file or directory: 'Amazon Sale Report.csv'

In [ ]:
#Distribution Of 'Amount' In Sales

# Plot histogram of 'Amount'
plt.hist(sales['Amount'], bins=50, color='blue', alpha=0.5, label='All Data')


#Find outliers of 'Amount'
from scipy.stats import zscore

sales['z_score'] = zscore(sales['Amount'])
outliers = sales[sales['z_score'].abs() > 3]

# Plot histogram of outliers against all data
plt.hist(outliers['Amount'], bins=50, color='red', alpha=0.7, label='Outliers')

plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.title('Comparison of Distribution: All Data vs Outliers')
plt.legend()
plt.show()



# Plot distribution with outliers
plt.figure(figsize=(12, 6))
sns.histplot(sales['Amount'], kde=True, bins=50)
plt.title('Distribution of Amount with Outliers')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.show()

# Outliers are those greater than 2000
filtered_sales = sales[sales['Amount'] <= 2000]


# Plot distribution without outliers
plt.figure(figsize=(12, 6))
sns.histplot(filtered_sales['Amount'], kde=True, bins=50)
plt.title('Distribution of Amount without Outliers')
plt.xlabel('Amount')
plt.ylabel('Frequency')
plt.show()










In [ ]:
#Total Sales Per Month


daily_sales =filtered_sales.groupby('Date')['Amount'].sum().reset_index()


# Get the min and max values for scaling
min_y_all = daily_sales['Amount'].min()
max_y_all = daily_sales['Amount'].max()

plt.figure(figsize=(12, 6))
plt.plot(daily_sales.index, daily_sales['Amount'], marker='o')
plt.title('Filltered Total Sales Per Day')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.ylim(0,  max_y_all+20000)  # Set y-axis limits
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


# Filter out sales data for March
sales_filtered = sales[~sales['Date'].dt.month.isin([3])]

# Group by month and sum the sales
monthly_sales_filtered = sales_filtered.groupby(sales_filtered['Date'].dt.to_period('M'))['Amount'].sum()

# Plot monthly sales excluding March since only one day in March data
plt.figure(figsize=(12, 6))
monthly_sales_filtered.plot(kind='bar')
plt.title('Filtered Total Sales Per Month (Excluding March)')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#SKU Exploration


#Top 20 SKUs
import seaborn as sns
import matplotlib.pyplot as plt

# Get the counts of each SKU
sku_counts = sales['SKU'].value_counts()

print(sales['SKU'].nunique())
print(sales['SKU'].describe())

# Filter for the top 20 SKUs if necessary
top_20_skus = sku_counts.head(20).reset_index()
top_20_skus.columns = ['SKU', 'Count']

# Plot the distribution
plt.figure(figsize=(12, 8))
sns.barplot(x='SKU', y='Count', data=top_20_skus, palette='viridis')
plt.title('Distribution of Top 20 SKUs')
plt.xlabel('SKU')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate SKU labels for better readability
plt.show()



#Simplifying SKUs By Eliminating Sizes

# Define a list of size suffixes to remove
size_suffixes = ['-XS','-S', '-M', '-L', '-XL', '-XXL','-XXXL']

# Remove size suffixes from the SKU column
def remove_size_suffix(sku):
    for suffix in size_suffixes:
        if sku.endswith(suffix):
            return sku[:-len(suffix)]
    return sku

sales['Simplified_SKU'] = sales['SKU'].apply(remove_size_suffix)

# Get the counts of each simplified SKU
simplified_sku_counts = sales['Simplified_SKU'].value_counts()

# Output the number of unique simplified SKUs
#print("Unique simplified SKU count:", len(simplified_sku_counts))

# Describe the distribution of simplified SKUs
#print(simplified_sku_counts.describe())

# Optionally, if you want to see the top simplified SKUs
top_simplified_skus = simplified_sku_counts.head(20).reset_index()
top_simplified_skus.columns = ['SKU', 'Count']


import seaborn as sns
import matplotlib.pyplot as plt

# Plot the distribution of the simplified SKUs
plt.figure(figsize=(14, 10))
sns.barplot(x='SKU', y='Count', data=top_simplified_skus, palette='viridis')
plt.title('Distribution of Top 20 Simplified SKUs')
plt.xlabel('SKU')
plt.ylabel('Count')
plt.xticks(rotation=90)  # Rotate SKU labels for better readability
plt.show()


print(top_simplified_skus)





In [ ]:
#SEGMENTATION BEGIN

# Define broader categories
category_mapping = {
    'Set': 'Accessories',
    'kurta': 'Ethnic Wear',
    'Western Dress': 'Western Wear',
    'Top': 'Western Wear',
    'Ethnic Dress': 'Ethnic Wear',
    'Bottom': 'Accessories',
    'Saree': 'Ethnic Wear',
    'Blouse': 'Accessories',
    'Dupatta': 'Ethnic Wear'
}

# Apply the mapping to create a new column for broader segments
sales['Segment'] = sales['Category'].map(category_mapping)

# Calculate total sales by segment
segment_sales = sales.groupby('Segment')['Amount'].sum()

# Plot the sales by segment
plt.figure(figsize=(10, 6))
sns.barplot(x=segment_sales.index, y=segment_sales.values)
plt.title('Total Sales by Segment')
plt.ylabel('Total Sales')
plt.xlabel('Segment')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Sample DataFrame for demonstration
# Replace this with your actual DataFrame
# sales = pd.DataFrame({'SKU': [...], 'Category': [...], 'Amount': [...]})

# Define a list of size suffixes to remove
size_suffixes = ['-XS','-S', '-M', '-L', '-XL', '-XXL','-XXXL']

# Function to remove size suffixes from SKUs
def remove_size_suffix(sku):
    for suffix in size_suffixes:
        if sku.endswith(suffix):
            return sku[:-len(suffix)]
    return sku

# Apply the function to create the 'Simplified_SKU' column
sales['Simplified_SKU'] = sales['SKU'].apply(remove_size_suffix)

# Group by 'Simplified_SKU' and calculate the total amount
sku_amounts = sales.groupby(['Simplified_SKU', 'Category'])['Amount'].sum().reset_index(name='Total_Amount')

# Get the top 20 SKUs by total amount
top_20_skus = sku_amounts.nlargest(20, 'Total_Amount')

# Create a color palette for the categories
palette = sns.color_palette("husl", len(top_20_skus['Category'].unique()))

# Plot the top 20 SKUs by total amount with colors representing categories
plt.figure(figsize=(14, 10))
sns.barplot(x='Total_Amount', y='Simplified_SKU', hue='Category', data=top_20_skus, palette=palette)

# Add titles and labels
plt.title('Top 20 Simplified SKUs by Total Amount (Categorized)')
plt.xlabel('Total Amount')
plt.ylabel('Simplified SKU')
plt.legend(title='Category', loc='right')

# Show the plot
plt.tight_layout()
plt.show()
